# This is a BAC tutorial
#### Enjoy!

## Import modules

In [ ]:
from bac.simulate.gromacs import (Simulation, Integrator, CenterOfMassMotion, CoulombType, CoulombModifierType, 
                                  VanDerWaalsModifierType, LongRangeDispersionCorrectionType, ConstraintType, 
                                  CouplingType, TemperatureCouplingType, PressureCouplingType)
from pathlib import Path

## STEP1: Minimize
The first step of our simulation is to minimize our system. The beauty of BAC is that once you create your
first simulation object, the next ones (see Step 2.) are very easy to creat.

In [2]:
minimize = Simulation(integrator=Integrator.steep)
minimize.delta_time = 0.002
minimize.number_of_steps = 2500
minimize.center_of_mass_motion = CenterOfMassMotion.linear

minimize.minimization_tolerance = 100

minimize.generate_velocities = True
minimize.generate_temperature = 298.15

minimize.energy_log_output_frequency = 100
minimize.recalculate_energies_frequency = 100
minimize.energy_output_frequency = 100


In [3]:
nbc = minimize.non_bonded_controller

nbc.cutoff = 1.2
nbc.coulomb_type = CoulombType.pme
nbc.coulomb_modifier = CoulombModifierType.potential_shift_verlet
nbc.coulomb_cutoff = 1.2

nbc.van_der_waals_modifier = VanDerWaalsModifierType.potential_switch
nbc.van_der_waals_switch_cutoff = 0.9
nbc.van_der_waals_cutoff = 1.2

nbc.correct_long_range_dispersion = LongRangeDispersionCorrectionType.energy_and_pressure

nbc.pme_order = 6
nbc.ewald_tolerance_coulomb = 1e-06

In [4]:
minimize.temperature_controller.time = 2.0
minimize.temperature_controller.temperature = 298.15

minimize.pressure_controller.compressibility = 4.5e-5
minimize.pressure_controller.pressure = 1.01325

In [2]:
minimize.constraints.type = ConstraintType.h_bonds
minimize.constraints.lincs_order = 12

In [6]:
minimize.free_energy_controller = gmx.FreeEnergyController()
fe = minimize.free_energy_controller
fe.couple_molecule_groups = 'mobley_9055303'
fe.couple_type_initial_lambda = CouplingType.vdw_and_coulomb
fe.couple_type_final_lambda = CouplingType.none
fe.initial_lambda_state = 0
fe.fep_lambdas = [0.0, 0.25, 0.5, 0.75] + [1.0]*16
fe.van_der_waals_lambdas = [0.0]*5 + [0.05, 0.10, 0.2, 0.30, 0.4, 0.50, 0.6, 0.65, 
                                      0.70, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
fe.calculate_lambda_neighbors = -1

fe.soft_core_alpha = 0.5
fe.soft_core_power = 1

In [7]:
minimize.coordinates = 'mobley_9055303.gro'
minimize.topology = 'mobley_9055303.top'
minimize.name = 'minimize'

# STEP2: NVT equilibration
Just call the `next(previous_step)` function to get the next step. This sets all input and outputs for you. Change the name and integrator.

In [8]:
equilibration_nvt = next(minimize)
equilibration_nvt.name = 'equilibration_nvt'
equilibration_nvt.integrator = Integrator.sd
equilibration_nvt.number_of_steps = 25000

# STEP 3: NPT eq

In [9]:
equilibration_npt = next(equilibration_nvt)
equilibration_npt.name = 'equilibration_npt_1'
equilibration_npt.temperature_controller.type = TemperatureCouplingType.berendsen
equilibration_npt.temperature_controller.time = 0.1
equilibration_npt.pressure_controller.type = PressureCouplingType.berendsen
equilibration_npt.generate_velocities = False
equilibration_npt.constraints.continuation = True

# STEP 4: NPT eq 2

In [10]:
equilibration_npt_2 = next(equilibration_npt)
equilibration_npt_2.name = 'equilibration_npt_2'
equilibration_npt_2.temperature_controller.type = TemperatureCouplingType.no
equilibration_npt_2.temperature_controller.time = 2.0
equilibration_npt_2.pressure_controller.type = PressureCouplingType.parrinello_rahman
equilibration_npt_2.pressure_controller.time = 10

# STEP 5: Production baby!

In [11]:
prod = next(equilibration_npt_2)
prod.name = 'production'
prod.number_of_steps = 250_000
prod.compressed_coordinate_output_frequency = 1000
prod.constraints.continuation = False

# Workflow

In [12]:
from bac.simulate import Workflow, Replica, LambdaWindow

In [13]:
wf = Workflow(resource='bluewaters', r_dir='mobley_9055303')

In [14]:
wf.add_simulation(minimize)
wf.add_simulation(equilibration_nvt)
wf.add_simulation(equilibration_npt)
wf.add_simulation(equilibration_npt_2)
wf.add_simulation(prod)

In [15]:
wf.ensembles = [Replica(number_of=1), LambdaWindow(number_of_states=2)]

In [16]:
wf.preprocess_simulations()